In [17]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def load_image(image_path):
    try:
        image = Image.open(image_path)

        image = np.array(image)
        if len(image.shape) == 3 and image.shape[2] == 4:
            # image = cv2.cvtColor(image, cv2.COlOR_RGB2BGR)
            #swap RGB to BGR while preserviing alpha channel.
            image = image[:,:,[2,1,0,3]]
        return image
    except Exception as e:
        print("error loading image " + image_path)
        raise e
    
def overlay_image(source, overlay, x_start,x_end,y_start,y_end):
    result = source.copy()

    target_height = y_end-y_start
    target_width = x_end-x_start

    resized_overlay = calc_dim_resize(overlay,target_height,target_width)
    print(f"resized_overlay shape: {resized_overlay.shape}")
    print(f"source shape: {source.shape}")

    if resized_overlay.shape[-1] == 4:  # Overlay has alpha channel
           overlay_bgr = resized_overlay[:, :, :3]
           overlay_alpha = resized_overlay[:, :, 3] / 255.0
    else:  # Overlay doesn't have alpha channel
        overlay_bgr = resized_overlay
        overlay_alpha = np.ones((target_height, target_width))
        
    # Get source alpha
    background_alpha = source[y_start:y_end, x_start:x_end, 3] / 255.0
    
    # Blend the images considering both alpha channels
    for c in range(3):  # Process BGR channels
        result[y_start:y_end, x_start:x_end, c] = \
            (1 - overlay_alpha) * source[y_start:y_end, x_start:x_end, c] + \
            overlay_alpha * overlay_bgr[:, :, c]
    
    # Compute final alpha channel
    result[y_start:y_end, x_start:x_end, 3] = \
    (overlay_alpha + background_alpha * (1 - overlay_alpha)) * 255
        
    return result

def display_image(image):
    plt.figure(figsize=(10,8))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def calc_dim_resize(image, target_height,target_width):
    resized_image = cv2.resize(image, (target_height,target_width))
    return resized_image

In [18]:
image_path = 'images/source_pic.png'
overlay_path = 'images/texas-cap.JPG'

source = load_image(image_path)
overlay = load_image(overlay_path)

y_start , y_end = 2, 62
x_start , x_end = 158, 330

result = overlay_image(source, overlay,y_start , y_end,x_start , x_end )

display_image(result)

resized_overlay shape: (60, 172, 3)
source shape: (499, 504, 4)


ValueError: operands could not be broadcast together with shapes (172,60) (60,172) 